In [173]:
import pandas as pd
import numpy as np
import duckdb
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import preprocessing
import statsmodels.api as sm

We previously dealt with the missing values by replacing them with the mean of their respective column. However, due to the significant number of missing values, this approach resulted in linear regression model that likely lacks accuracy. As an alternative approach, we can consider running a linear regression model using the average value of each variable aggregated across all years for each country. 
To do this, we must first create a dataframe that contains the average value of each variable aggregated across all years for each country.

In [174]:
combined_df = pd.read_csv('combined_data.csv')

In [175]:
combined_df.head()

,Country,Year,GDP per Capita,Population(Million),Average Spending on Higher Education (USD/student),Government Expenditure On Education (%),Government Spending Teritary (% Gov Spending),Household Income per Capita,Number of Universities,Primary Enrollment rate (% gross),Secondary Enrollment rate (% gross),Tertiary Enrollment rate (% gross),Population with Tertiary Education (%)
0,AUS,1995,22442.32,18.004882,NaN,5.13413,1.4,NaN,187,101.29127,143.23387,70.68857,24.697142
1,AUS,1996,23289.40,18.224767,NaN,5.23100,1.6,NaN,187,101.58029,148.63600,75.64778,NaN
2,AUS,1997,24477.41,18.423037,NaN,NaN,..,NaN,187,101.18403,152.93597,80.90665,25.702168
3,AUS,1998,25708.26,18.607584,NaN,NaN,..,NaN,187,100.92862,NaN,NaN,28.139578
4,AUS,1999,27139.25,18.812264,NaN,NaN,..,NaN,187,100.65884,NaN,NaN,28.981586


In [176]:
combined_df.dtypes

Country                                                object
Year                                                    int64
GDP per Capita                                        float64
Population(Million)                                   float64
Average Spending on Higher Education (USD/student)    float64
Government Expenditure On Education (%)               float64
Government Spending Teritary (% Gov Spending)          object
Household Income per Capita                           float64
Number of Universities                                  int64
Primary Enrollment rate (% gross)                     float64
Secondary Enrollment rate (% gross)                   float64
Tertiary Enrollment rate (% gross)                    float64
Population with Tertiary Education (%)                float64
dtype: object

In [177]:
combined_df["Government Spending Teritary (% Gov Spending)"] = pd.to_numeric(combined_df["Government Spending Teritary (% Gov Spending)"], errors="coerce")

In [178]:
combined_df.dtypes

Country                                                object
Year                                                    int64
GDP per Capita                                        float64
Population(Million)                                   float64
Average Spending on Higher Education (USD/student)    float64
Government Expenditure On Education (%)               float64
Government Spending Teritary (% Gov Spending)         float64
Household Income per Capita                           float64
Number of Universities                                  int64
Primary Enrollment rate (% gross)                     float64
Secondary Enrollment rate (% gross)                   float64
Tertiary Enrollment rate (% gross)                    float64
Population with Tertiary Education (%)                float64
dtype: object

In [179]:
query = """
        SELECT 
            Country,
            AVG("GDP per capita") AS "GDP per Capita",
            AVG("Population(Million)") AS "Population (Million)",
            AVG("Average Spending on Higher Education (USD/student)")
            AS "Average Spending on Higher Education (USD/student)",
            AVG("Government Expenditure On Education (%)")
            AS "Government Expenditure On Education (%)",
            AVG("Government Spending Teritary (% Gov Spending)")
            AS "Government Spending Teritary (% Gov Spending)",
            AVG("Household Income per Capita")
            AS "Household Income per Capita",
            AVG("Number of Universities")
            AS "Number of Universities",
            AVG("Primary Enrollment rate (% gross)")
            AS "Primary Enrollment rate (% gross)",
            AVG("Secondary Enrollment rate (% gross)")
            AS "Secondary Enrollment rate (% gross)",
            AVG("Tertiary Enrollment rate (% gross)")
            AS "Tertiary Enrollment rate (% gross)",
            AVG("Population with Tertiary Education (%)")
            AS "Population with Tertiary Education (%)"
        FROM combined_df
        GROUP BY Country
        
        """
df = duckdb.sql(query).df()
df.head()

,Country,GDP per Capita,Population (Million),Average Spending on Higher Education (USD/student),Government Expenditure On Education (%),Government Spending Teritary (% Gov Spending),Household Income per Capita,Number of Universities,Primary Enrollment rate (% gross),Secondary Enrollment rate (% gross),Tertiary Enrollment rate (% gross),Population with Tertiary Education (%)
0,AUS,38143.8412,21.221520,17397.198571,5.063198,1.243750,35349.783486,187.0,102.619088,147.339639,100.465227,40.367922
1,AUT,39269.2264,8.311855,18464.942500,5.474380,1.500000,33898.521290,84.0,101.832554,100.166013,70.926428,35.767432
2,BEL,36588.5528,10.709564,17163.086154,5.935744,1.275000,31524.486336,142.0,102.601560,155.965544,65.986742,41.729849
3,CAN,37217.9484,33.088363,22755.057500,5.031936,1.733333,30849.148394,383.0,100.065320,106.513946,67.084939,53.696534
4,CHE,49494.9216,7.678109,24848.250000,4.900874,1.245455,38082.094754,103.0,103.458437,97.630051,48.298670,35.897165


In [180]:
df[df.isna().any(axis=1)]

,Country,GDP per Capita,Population (Million),Average Spending on Higher Education (USD/student),Government Expenditure On Education (%),Government Spending Teritary (% Gov Spending),Household Income per Capita,Number of Universities,Primary Enrollment rate (% gross),Secondary Enrollment rate (% gross),Tertiary Enrollment rate (% gross),Population with Tertiary Education (%)
6,COL,10366.1188,42.574965,5183.988900,4.300569,0.870588,NaN,299.0,118.238453,87.256054,36.675314,26.549254
19,ISL,39542.5520,0.306963,11913.987571,6.989704,1.319048,NaN,9.0,99.182653,111.354106,64.206610,36.515743
20,ISR,29219.7068,7.243359,11648.622800,5.940658,1.000000,NaN,55.0,105.379374,103.272581,58.582081,44.532799


Looking at the dataframe, we noticed that there are three missing values in the 'Household Income per Capita' column. We attempted to find data on household income per capita for these countries so we could manually input them, but were unsuccessful. Therefore, our only option was to remove the data for these countries. 

In [181]:
df = df[(df["Country"] != "ISL") & (df["Country"] != "ISR") & (df["Country"] != "COL")]
df.reset_index(drop=True, inplace=True)
df[df.isna().any(axis=1)]

,Country,GDP per Capita,Population (Million),Average Spending on Higher Education (USD/student),Government Expenditure On Education (%),Government Spending Teritary (% Gov Spending),Household Income per Capita,Number of Universities,Primary Enrollment rate (% gross),Secondary Enrollment rate (% gross),Tertiary Enrollment rate (% gross),Population with Tertiary Education (%)


In [182]:
df.to_csv('country_average.csv', index=False)

Now that the missing values have been removed, we can run linear regression models on the data. We will first run a linear regression model on all the input variables. Before creating the model, we will first normalize the input variables. Normalization ensures that all input features have the same scale. This is important because linear regression is sensitive to the scale of input features. Features with larger scales may have a disproportionately larger impact on the model, potentially overshadowing the contributions of features with smaller scales. To do this, we will create a function Normalizer that takes a set of columns and normalizes them using 'StandardScaler' from scikit-learn, and apply the function to the input variables. StandardScaler standardizes the features by removing the mean and scaling to unit variance. 

Then, we will split the data into training and testing sets. We will use the training set to train the model and the testing set to evaluate its performance. Following convention, the test_size parameter is set to 30%, meaning 70% of the data will be used for training and 30% for testing. The 'random_state' parameter ensures reproducibility.

In [183]:
input_vars = ["GDP per Capita", "Population (Million)", "Average Spending on Higher Education (USD/student)", \
              "Government Expenditure On Education (%)", "Government Spending Teritary (% Gov Spending)",\
              "Household Income per Capita", "Number of Universities", "Primary Enrollment rate (% gross)", \
              "Secondary Enrollment rate (% gross)", "Tertiary Enrollment rate (% gross)"]
def Normalizer(df_cols):
    scaler = preprocessing.StandardScaler().fit(df_cols)
    return(scaler.transform(df_cols))
x = Normalizer(df[input_vars].values)
y = df["Population with Tertiary Education (%)"]
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=2950)
model = LinearRegression().fit(X_train, y_train)

Next, we will calculate the mean squared error (MSE) and root mean squared error (RMSE) on the testing and training data. These metrics help to evaluate how well the model's predictions align with the actual values in the dataset. A lower MSE and RMSE indicates better model performance.

In [184]:
y_test_pred = model.predict(X_test)
y_train_pred = model.predict(X_train)

mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
rmse_train = np.sqrt(mse_train)
rmse_test = np.sqrt(mse_test)

print("Mean Squared Error on Training Data: " + str(mse_train))
print("Mean Squared Error on Testing data: " + str(mse_test))
print("Root Mean Squared Error on Training Data: " + str(rmse_train))
print("Root Mean Squared Error on Testing Data: " + str(rmse_test))

Mean Squared Error on Training Data: 30.336330076317736
Mean Squared Error on Testing data: 102.69656515821113
Root Mean Squared Error on Training Data: 5.507842597271434
Root Mean Squared Error on Testing Data: 10.13393137722035


The model appears to perform well on the training data, as evidenced by the low MSE and RMSE. However, there is a noticeable increase in both MSE and RMSE when evaluating the model on the testing data, indicating a potential lack of generalization. The model may be overfitting to the training data, as it does not generalize as well to new, unseen data.

We will again model the percentage of the population with tertiary education using all input variables, but this time using OLS regression to find the p-value for each input variable and assess whether certain variables may be significant predictors of the percentage of the population with tertiary education. 

In [185]:
input_vars = ["GDP per Capita", "Population (Million)", "Average Spending on Higher Education (USD/student)", \
              "Government Expenditure On Education (%)", "Government Spending Teritary (% Gov Spending)",\
              "Household Income per Capita", "Number of Universities", "Primary Enrollment rate (% gross)", \
              "Secondary Enrollment rate (% gross)", "Tertiary Enrollment rate (% gross)"]
def Normalizer(df_cols):
    scaler = preprocessing.StandardScaler().fit(df_cols)
    return(scaler.transform(df_cols))
x = Normalizer(df[input_vars].values)
y = df["Population with Tertiary Education (%)"]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                                      OLS Regression Results                                      
Dep. Variable:     Population with Tertiary Education (%)   R-squared:                       0.601
Model:                                                OLS   Adj. R-squared:                  0.435
Method:                                     Least Squares   F-statistic:                     3.614
Date:                                    Wed, 15 Nov 2023   Prob (F-statistic):            0.00478
Time:                                            17:55:38   Log-Likelihood:                -115.38
No. Observations:                                      35   AIC:                             252.8
Df Residuals:                                          24   BIC:                             269.9
Df Model:                                              10                                         
Covariance Type:                                nonrobust                                         
          

Analyzing the p-values from the model, it suggests that the variables corresponding to "Average Spending on Higher Education (USD/student)" (x3) and "Tertiary Enrollment rate (% gross)" (x10) exhibit statistical significance at the 5% level. However, the insignificance of "GDP per Capita" (x1) may be influenced by the presence of noise and the considerable number of variables, potentially diluting the explained variance associated with GDP per capita. To better comprehend the relationship between GDP per capita and the percentage of the population with tertiary education and mitigate the impact of other factors, we will create another regression model using only GDP per capita as our input variable. We will do the same thing with variables x3 and x10 and compare their AIC values.

In [187]:
x = Normalizer(df["GDP per Capita"].values.reshape(-1, 1))
y = df["Population with Tertiary Education (%)"]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                                      OLS Regression Results                                      
Dep. Variable:     Population with Tertiary Education (%)   R-squared:                       0.270
Model:                                                OLS   Adj. R-squared:                  0.248
Method:                                     Least Squares   F-statistic:                     12.24
Date:                                    Wed, 15 Nov 2023   Prob (F-statistic):            0.00136
Time:                                            17:55:38   Log-Likelihood:                -125.93
No. Observations:                                      35   AIC:                             255.9
Df Residuals:                                          33   BIC:                             259.0
Df Model:                                               1                                         
Covariance Type:                                nonrobust                                         
          

In [188]:
x = Normalizer(df["Average Spending on Higher Education (USD/student)"].values.reshape(-1, 1))
y = df["Population with Tertiary Education (%)"].values.reshape(-1, 1)
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.238
Model:                            OLS   Adj. R-squared:                  0.215
Method:                 Least Squares   F-statistic:                     10.31
Date:                Wed, 15 Nov 2023   Prob (F-statistic):            0.00295
Time:                        17:55:38   Log-Likelihood:                -126.69
No. Observations:                  35   AIC:                             257.4
Df Residuals:                      33   BIC:                             260.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         35.1974      1.572     22.383      0.0

In [189]:
x = Normalizer(df["Tertiary Enrollment rate (% gross)"].values.reshape(-1, 1))
y = df["Population with Tertiary Education (%)"]
x = sm.add_constant(x)
model = sm.OLS(y, x).fit()
print(model.summary())

                                      OLS Regression Results                                      
Dep. Variable:     Population with Tertiary Education (%)   R-squared:                       0.140
Model:                                                OLS   Adj. R-squared:                  0.114
Method:                                     Least Squares   F-statistic:                     5.390
Date:                                    Wed, 15 Nov 2023   Prob (F-statistic):             0.0266
Time:                                            17:55:38   Log-Likelihood:                -128.81
No. Observations:                                      35   AIC:                             261.6
Df Residuals:                                          33   BIC:                             264.7
Df Model:                                               1                                         
Covariance Type:                                nonrobust                                         
          

Comparing the three variables' AIC values, the variable 'GDP per Capita' has the lowest AIC at 255.9. This suggests that in the context of explaining the variation in the percentage of the population with tertiary education, the model with 'GDP per Capita' as the sole predictor provides the best balance between goodness of fit and model complexity. This suggests support for hypothesis 1, which posits that GDP has a greater impact on completed tertiary education rates compared to other factors in our model.